In [1]:
import sys
sys.path.append("../../")
from STK_Sim import *

Filename = 'AERO_402_Further_Assessment'

stk_object = STK_Simulation(False,Filename)

In [2]:
# Setting scenario time variables
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "UTCG")
start_time = time_convert(stk_object.root.CurrentScenario.StartTime)
duration = datetime.timedelta(days=1, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
stop_time=(start_time+duration).strftime("%d %b %Y %H:%M:%S.%f")
stk_object.root.CurrentScenario.StopTime=stop_time
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "EpSec")

# All of these variables substantially change calculation time
stk_object.dt = 30
n_targets = 15

In [3]:
# Generating Targets

# Generating a polygon to bound the lat/lon coordinates, you can create your polygon, in the same format as Targets_Polygon.txt.
poly = Create_Poly('../../Input_Files/Targets_Polygon.txt')

# Writing random points within the polygon to a target file.
targets_filename = '../../Input_Files/Targets_File.txt'
# polygon_random_points(poly,n_targets,targets_filename)

# Loading targets into stk from file.
stk_object.Target_Loader(targets_filename)

# # Plotting the polygon and generated targets on the map.
# plot_targets_and_polygon(poly,targets_filename)

In [4]:
stk_object.Satellite_Loader("../../Input_Files/Satellites_File.txt")

In [7]:
stk_object.dt = 30
stk_object.Interpolate = True
stk_object.Compute_AzEl()

- Computing_AzEl [          ] ▁▃▅ 0/150 [0%] in 0s (~0s, 0.0/s) 

- Computing_AzEl [==========] 150/150 [100%] in 3.9s (38.72/s)                  


0